In [ ]:
%pip install keras_facenet

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import cv2
import numpy as np
import os

from keras_facenet import FaceNet
from sklearn.model_selection import train_test_split
import pickle

In [11]:
persons = ['Karolina','Kacper','Joanna','Mirek','Sylwia','Łukasz','Emilia','Mariusz','Ania'] ## classes

In [ ]:
path_to_detected_faces_BGR = '/content/drive/MyDrive/Detected_Faces_BGR'
def convert_to_BGR(folder_path, data_type, name):
  path_save_path = os.path.join(path_to_detected_faces_BGR, data_type, name)
  os.makedirs(path_save_path, exist_ok=True)

  saved_count = 0
  for image_path in os.listdir(folder_path):
    img = cv2.imread(os.path.join(folder_path, image_path))
    bgr_image = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
    save_path = os.path.join(path_save_path,f"cropped_{saved_count:04d}.jpg")
    cv2.imwrite(save_path, bgr_image)
    saved_count +=1

In [ ]:
path_to_no_covering_data = '/content/drive/MyDrive/Detected_Faces/no_covering'
for name in persons:
    for i in range(1,4):
        convert_to_BGR(f'{path_to_no_covering_data}/{name}/{name}_{i}', 'no_covering', f'{name}/{name}_{i}')

In [ ]:
path_to_no_covering_data = '/content/drive/MyDrive/Detected_Faces/sunglasses'
for name in persons:
    for i in range(1,4):
        convert_to_BGR(f'{path_to_no_covering_data}/{name}/{name}_{i}', 'sunglasses', f'{name}/{name}_{i}')

In [ ]:
embedder = FaceNet()

def get_embeddings(face_img_path):
    '''
    Function that returns embeddings of face image
    :param face_img_path: path to face image
    :return: embeddings of face image
    '''
    face_img = cv2.imread(face_img_path)
    face_img = face_img.astype('float32')
    face_img = np.expand_dims(face_img, axis=0)
    yhat = embedder.embeddings(face_img)
    return yhat[0]

##### Extraction of facial features without a cover

In [ ]:
DETECTED_FACES_MAIN_PATH = '/content/drive/MyDrive/Detected_Faces_BGR/no_covering'

embeddings_dict = {f"{name}_{i}": [] for name in persons for i in range(1, 4)}

for name in persons:
    for i in range(1, 4):
        folder_name = f"{name}_{i}"
        print(f"Processing images in folder: {folder_name}")
        
        path = os.path.join(DETECTED_FACES_MAIN_PATH, name, folder_name)
        if not os.path.exists(path):
            print(f"Folder not found: {path}")
            continue
        
        for image_path in os.listdir(path):
            embedding = get_embeddings(os.path.join(path, image_path))
            embeddings_dict[folder_name].append(embedding)

##### Division of data into training and testing

In [7]:
X_train, X_test = [], []
y_train, y_test = [], []

for folder, embeddings in embeddings_dict.items():
    label = folder.split('_')[0]
    
    train, test = train_test_split(embeddings, test_size=0.2, random_state=42)
    
    X_train.extend(train)
    y_train.extend([label] * len(train))
    X_test.extend(test)
    y_test.extend([label] * len(test))

In [ ]:
print(len(X_train), len(y_train))
print(len(X_test), len(y_test))

##### Data saving

In [ ]:
trainX_path = '/content/drive/MyDrive/trainX.pkl'
trainy_path = '/content/drive/MyDrive/trainy.pkl'
testX_path = '/content/drive/MyDrive/testX.pkl'
testy_path = '/content/drive/MyDrive/testy.pkl'

with open(trainX_path, 'wb') as file:
    pickle.dump(X_train, file)
with open(trainy_path, 'wb') as file:
    pickle.dump(y_train, file)
with open(testX_path, 'wb') as file:
    pickle.dump(X_test, file)
with open(testy_path, 'wb') as file:
    pickle.dump(y_test, file)

##### Extraction of facial features with sunglasses

In [ ]:
DETECTED_FACES_MAIN_PATH = '/content/drive/MyDrive/Detected_Faces_BGR/sunglasses'
SAVE_MAIN_PATH = '/content/drive/MyDrive/'

def process_and_save_data(main_path, save_path, persons):
    for name in persons:
        print(f"Processing data for: {name}")
        
        person_embeddings = []
        
        for i in range(1, 4): 
            folder_name = f"{name}_{i}"
            path = os.path.join(main_path, name, folder_name)
            
            if not os.path.exists(path):
                print(f"Folder not found: {path}")
                continue
            
            for image_path in os.listdir(path):
                embedding = get_embeddings(os.path.join(path, image_path))
                person_embeddings.append(embedding)
        
        file_path = os.path.join(save_path, f"testX_{name.lower()}.pkl")
        with open(file_path, 'wb') as file:
            pickle.dump(person_embeddings, file)
        print(f"Saved data for {name} to {file_path}")

process_and_save_data(DETECTED_FACES_MAIN_PATH, SAVE_MAIN_PATH, persons)